In [ ]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

In [ ]:
tensor_transform = get_tensor_transform('ImageNet', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
df_train = get_df(df, 20, False, True, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
train_loader = get_loader(16, 1, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(16, 1, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx = get_indices(df_test, root_dir)
test_loader = get_loader(16, 1, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, False)

torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import precision_score,f1_score, accuracy_score, jaccard_score

In [ ]:
inputs, classes = next(iter(train_loader))

In [ ]:
inputs.shape

In [ ]:
show_batch(train_loader, 1)

In [ ]:
resnet.eval()
running_loss = 0.0
running_acc = 0.0  
running_f1 = 0.0
valid_result = []
rolling_preds = np.zeros(5)
y_pred = []
y_true = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.cuda()
        y = Variable(y.float()).cuda()
        X = X.squeeze(dim=0)
        X = X.permute(1,0,2,3)
        y = y.squeeze(dim=0)
        y = y.float()
        output = resnet(X)
        preds = torch.sigmoid(output) #.data > 0.5  
        preds = preds.to(torch.float32)  
        preds = preds.detach().cpu().numpy()
        cl1 = []
        cl2 = []
        cl3 = []
        cl4 = []
        cl5 = []
        for p in preds:
            cl1.append(p[0])
            cl2.append(p[1])
            cl3.append(p[2])
            cl4.append(p[3])
            cl5.append(p[4])

        if (np.mean(cl1) > 0.5):
            rolling_preds[0] = 1
        else:
            rolling_preds[0] = 0

        if (np.mean(cl2) > 0.7):
            rolling_preds[1] = 1
        else:
            rolling_preds[1] = 0   

        if (np.mean(cl3) > 0.3):
            rolling_preds[2] = 1
        else:
            rolling_preds[2] = 0

        if (np.mean(cl4) > 0.8):
            rolling_preds[3] = 1
        else:
            rolling_preds[3] = 0 

        if (np.mean(cl5) > 0.8):
            rolling_preds[4] = 1
        else:
            rolling_preds[4] = 0        

        #running_acc += accuracy_score(y.detach().cpu().numpy(), rolling_preds) *  X.size(0)
        #running_f1 += f1_score(y.detach().cpu().numpy(), (rolling_preds), average="samples")  *  X.size(0)
        y_pred.append(rolling_preds)
        y_true.append(y.detach().cpu().numpy())

# epoch_acc = running_acc / len(valid_loader.dataset)
# epoch_f1 = running_f1 / len(valid_loader.dataset)

# valid_result.append('Validation Acc: {:.4f} F1: {:.4f}'.format(epoch_acc, epoch_f1))
# print(valid_result)

In [ ]:
from helpers_thresholds import *

In [ ]:
rolling_preds

In [ ]:
y_true

In [ ]:
y_pred

In [ ]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [ ]:
thresholds = nsea_compute_thresholds(y_true, y_pred, classes)

In [ ]:
thresholds

## validation set

In [ ]:
new_compute_metrics(y_tr, y_pr, thresholds, classes)

## test set

In [ ]:
resnet.eval()
running_loss = 0.0
running_acc = 0.0  
running_f1 = 0.0
valid_result = []
rolling_preds = np.zeros(5)
y_pred = []
y_true = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.cuda()
        y = Variable(y.float()).cuda()
        X = X.squeeze(dim=0)
        X = X.permute(1,0,2,3)
        y = y.squeeze(dim=0)
        y = y.float()
        output = resnet(X)
        preds = torch.sigmoid(output) #.data > 0.5  
        preds = preds.to(torch.float32)  
        preds = preds.detach().cpu().numpy()
        cl1 = []
        cl2 = []
        cl3 = []
        cl4 = []
        cl5 = []
        for p in preds:
            cl1.append(p[0])
            cl2.append(p[1])
            cl3.append(p[2])
            cl4.append(p[3])
            cl5.append(p[4])

        if (np.mean(cl1) > 0.5):
            rolling_preds[0] = 1
        else:
            rolling_preds[0] = 0

        if (np.mean(cl2) > 0.5):
            rolling_preds[1] = 1
        else:
            rolling_preds[1] = 0   

        if (np.mean(cl3) > 0.5):
            rolling_preds[2] = 1
        else:
            rolling_preds[2] = 0

        if (np.mean(cl4) > 0.5):
            rolling_preds[3] = 1
        else:
            rolling_preds[3] = 0 

        if (np.mean(cl5) > 0.5):
            rolling_preds[4] = 1
        else:
            rolling_preds[4] = 0        

        #running_acc += accuracy_score(y.detach().cpu().numpy(), rolling_preds) *  X.size(0)
        #running_f1 += f1_score(y.detach().cpu().numpy(), (rolling_preds), average="samples")  *  X.size(0)
        y_pred.append(rolling_preds)
        y_true.append(y.detach().cpu().numpy())

# epoch_acc = running_acc / len(valid_loader.dataset)
# epoch_f1 = running_f1 / len(valid_loader.dataset)

# valid_result.append('Validation Acc: {:.4f} F1: {:.4f}'.format(epoch_acc, epoch_f1))
# print(valid_result)

In [ ]:
new_compute_metrics(y_tr, y_pr, thresholds, classes)